In [136]:
import pandas as pd
import numpy as np
import numpy as np,pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
import joblib
from sklearn.impute import SimpleImputer



In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


PREPROCESSING THE DATA

In [138]:
first_df=pd.read_csv('/content/drive/MyDrive/AI Projects/players_21.csv')

In [139]:
first_df

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,https://sofifa.com/player/257710/mengxuan-zhan...,Zhang Mengxuan,张梦炫,CB,47,52,70000.0,1000.0,21,...,47+2,47+2,47+2,45+2,15+2,https://cdn.sofifa.net/players/257/710/21_120.png,https://cdn.sofifa.net/teams/112165/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
18940,257933,https://sofifa.com/player/257933/wenzhuo-huang...,Huang Wenzhou,黄文卓,CM,47,53,70000.0,1000.0,21,...,46+2,46+2,46+2,47+2,15+2,https://cdn.sofifa.net/players/257/933/21_120.png,https://cdn.sofifa.net/teams/112540/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
18941,257936,https://sofifa.com/player/257936/yue-song/210002,Song Yue,宋岳,CM,47,47,45000.0,2000.0,28,...,46+1,46+1,46+1,47,11+2,https://cdn.sofifa.net/players/257/936/21_120.png,https://cdn.sofifa.net/teams/111774/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
18942,258736,https://sofifa.com/player/258736/vani-da-silva...,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,47,67,130000.0,500.0,17,...,26+2,26+2,26+2,33+2,14+2,https://cdn.sofifa.net/players/258/736/21_120.png,https://cdn.sofifa.net/teams/1920/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/gb-eng.png


In [140]:
first_df.isnull().sum()

sofifa_id               0
player_url              0
short_name              0
long_name               0
player_positions        0
                    ...  
player_face_url         0
club_logo_url         225
club_flag_url         225
nation_logo_url     17817
nation_flag_url         0
Length: 110, dtype: int64

In [141]:
label_encoder = LabelEncoder()
first_df['player_positions'] = label_encoder.fit_transform(first_df['player_positions'])

In [142]:
first_df['player_positions'].head

<bound method NDFrame.head of 0        539
1        587
2        560
3        331
4         10
        ... 
18939     55
18940    149
18941    149
18942    560
18943    149
Name: player_positions, Length: 18944, dtype: int64>

In [143]:
correlation_with_overall = first_df.corrwith(first_df['overall'])
sorted_correlations = correlation_with_overall.sort_values(ascending=False)
sorted_correlations[:35]

<ipython-input-143-9fba18009bb6>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_with_overall = first_df.corrwith(first_df['overall'])


overall                        1.000000
movement_reactions             0.867234
passing                        0.711521
mentality_composure            0.705252
dribbling                      0.641097
potential                      0.636366
release_clause_eur             0.626277
wage_eur                       0.592452
value_eur                      0.558493
power_shot_power               0.558372
physic                         0.530386
mentality_vision               0.509087
attacking_short_passing        0.502191
goalkeeping_speed              0.497350
shooting                       0.488315
skill_long_passing             0.487147
age                            0.468197
skill_ball_control             0.449372
international_reputation       0.440987
skill_curve                    0.420495
attacking_crossing             0.410530
power_long_shots               0.407525
mentality_aggression           0.401920
skill_fk_accuracy              0.385617
power_stamina                  0.381869


In [144]:
best_features=first_df[['overall', 'potential','mentality_composure', 'mentality_vision','passing', 'dribbling', 'release_clause_eur', 'wage_eur', 'value_eur',
'power_shot_power', 'shooting', 'power_long_shots','physic','age','international_reputation','attacking_short_passing', 'skill_long_passing', 'skill_ball_control', 'skill_fk_accuracy',
'skill_moves', 'skill_dribbling', 'skill_curve', 'attacking_crossing', 'attacking_volleys', 'attacking_heading_accuracy', 'attacking_finishing',
'movement_reactions','power_strength', 'power_stamina','mentality_positioning', 'mentality_penalties','defending', 'defending_marking_awareness']]

In [145]:
df=pd.DataFrame(best_features)

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   mentality_composure          18944 non-null  int64  
 3   mentality_vision             18944 non-null  int64  
 4   passing                      16861 non-null  float64
 5   dribbling                    16861 non-null  float64
 6   release_clause_eur           17949 non-null  float64
 7   wage_eur                     18719 non-null  float64
 8   value_eur                    18707 non-null  float64
 9   power_shot_power             18944 non-null  int64  
 10  shooting                     16861 non-null  float64
 11  power_long_shots             18944 non-null  int64  
 12  physic                       16861 non-null  float64
 13  age             

In [147]:
#Replacing all NaN values with mean
imp = SimpleImputer(strategy='mean')
imp.fit(df)
imputed_data=imp.fit_transform(df)
df=pd.DataFrame(imputed_data, columns=df.columns)

# Replace 93 overall with 92 if they appear only once
unique_93_overall = df['overall'].value_counts()[93]
if unique_93_overall == 1:
    df['overall'].replace(93, 92, inplace=True)



In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  float64
 1   potential                    18944 non-null  float64
 2   mentality_composure          18944 non-null  float64
 3   mentality_vision             18944 non-null  float64
 4   passing                      18944 non-null  float64
 5   dribbling                    18944 non-null  float64
 6   release_clause_eur           18944 non-null  float64
 7   wage_eur                     18944 non-null  float64
 8   value_eur                    18944 non-null  float64
 9   power_shot_power             18944 non-null  float64
 10  shooting                     18944 non-null  float64
 11  power_long_shots             18944 non-null  float64
 12  physic                       18944 non-null  float64
 13  age             

ENGINEERING THE FEATURES

In [149]:
skill_feats = ['passing','dribbling','release_clause_eur','wage_eur',
'value_eur']
df['skills'] = df[skill_feats].mean(axis=1)
df.drop(columns=skill_feats, inplace=True)

shooting_feats = ['power_shot_power','shooting','power_long_shots']
df['shotlevel'] = df[shooting_feats].mean(axis=1)
df.drop(columns=shooting_feats, inplace=True)

passing_feats=['attacking_short_passing','skill_long_passing','skill_ball_control',
'skill_fk_accuracy']
df['passing']=df[passing_feats].mean(axis=1)
df.drop(columns=passing_feats,inplace=True)


movement_feats=['skill_moves','skill_dribbling','skill_curve','attacking_crossing',
'attacking_volleys','attacking_heading_accuracy','attacking_finishing']
df['movements']=df[movement_feats].mean(axis=1)
df.drop(columns=movement_feats,inplace=True)

mentality_feats=['mentality_composure','mentality_vision']
df['mentality']=df[mentality_feats].mean(axis=1)
df.drop(columns=mentality_feats,inplace=True)

power_feats=['power_strength','power_stamina']
df['power']=df[power_feats].mean(axis=1)
df.drop(columns=power_feats,inplace=True)

positioning_feats=['mentality_positioning',
'mentality_penalties']
df['power']=df[positioning_feats].mean(axis=1)
df.drop(columns=positioning_feats,inplace=True)



In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  float64
 1   potential                    18944 non-null  float64
 2   physic                       18944 non-null  float64
 3   age                          18944 non-null  float64
 4   international_reputation     18944 non-null  float64
 5   movement_reactions           18944 non-null  float64
 6   defending                    18944 non-null  float64
 7   defending_marking_awareness  18944 non-null  float64
 8   skills                       18944 non-null  float64
 9   shotlevel                    18944 non-null  float64
 10  passing                      18944 non-null  float64
 11  movements                    18944 non-null  float64
 12  mentality                    18944 non-null  float64
 13  power           

In [158]:
features_correlations = df.corrwith(df['overall']).sort_values(ascending=False)
features_correlations

overall                        1.000000
movement_reactions             0.867232
mentality                      0.658147
potential                      0.636358
skills                         0.592173
shotlevel                      0.504925
physic                         0.493553
passing                        0.492889
age                            0.468198
international_reputation       0.440919
movements                      0.428057
power                          0.359592
defending                      0.333632
defending_marking_awareness    0.321001
dtype: float64

In [159]:
y=df['overall']
X=df.drop('overall',axis=1)

In [160]:
sc=StandardScaler()
scaled=sc.fit_transform(X)
X=pd.DataFrame(scaled, columns=X.columns)

In [161]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.1,random_state=42,stratify = y)

### USING RANDOM FOREST REGRESSOR

In [162]:
rf=RandomForestRegressor()

rf.fit(Xtrain, Ytrain)

y_pred=rf.predict(Xtest)

mean_absolute_error(y_pred,Ytest)

0.4542796833773089

### USING GRADIENT BOOSTING REGRESSOR

In [163]:
gb=GradientBoostingRegressor()

gb.fit(Xtrain, Ytrain)

y_pred_1 = gb.predict(Xtest)

mean_absolute_error(y_pred_1, Ytest)

0.7482111243922295

### USING XGBOOSTING REGRESSOR

In [164]:
x = XGBRegressor()

x.fit(Xtrain, Ytrain)

y_pred_2=x.predict(Xtest)

mean_absolute_error(y_pred_2,Ytest)

0.5230466857749113

### USING VOTING REGRESSOR ENSEMBLE MODEL

In [165]:
ensemble = VotingRegressor(estimators=[('rf', rf), ('xgb', x),('gb',gb)])

ensemble.fit(Xtrain, Ytrain)

y_pred_3=ensemble.predict(Xtest)

mean_absolute_error(y_pred_3, Ytest)

0.5157618558204965

### CROSS VALIDATION

In [166]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
'n_estimators': [100, 200, 300, 400, 500],
'learning_rate': [0.3, 0.1, 0.03],
'max_depth': [2,5, 6, 12]
  }
random_search = RandomizedSearchCV(estimator=x,param_distributions=param_dist,n_iter=50,  scoring='neg_mean_squared_error',cv=5, n_jobs=-1
)

random_search.fit(Xtrain, Ytrain)


KeyboardInterrupt: ignored

In [120]:
bm= random_search.best_estimator_

In [121]:
y_pred_4=bm.predict(Xtest)

In [122]:
mean_absolute_error(y_pred_4, Ytest)

0.48553203089570623

THE OVERALL BEST MODEL IS THE bm MODEL






### TESTING WITH FIFA22 DATASET

In [123]:
df_2=pd.read_csv('/content/drive/MyDrive/AI Projects/players_22 (1).csv',usecols=['overall', 'potential','mentality_composure', 'mentality_vision','passing', 'dribbling', 'release_clause_eur', 'wage_eur', 'value_eur',
'power_shot_power', 'shooting', 'power_long_shots','physic','age','international_reputation','attacking_short_passing', 'skill_long_passing', 'skill_ball_control', 'skill_fk_accuracy',
'skill_moves', 'skill_dribbling', 'skill_curve', 'attacking_crossing', 'attacking_volleys', 'attacking_heading_accuracy', 'attacking_finishing',
'movement_reactions','power_strength', 'power_stamina','mentality_positioning', 'mentality_penalties','defending', 'defending_marking_awareness' ])

In [124]:
df_2

,overall,potential,value_eur,wage_eur,age,skill_moves,international_reputation,release_clause_eur,shooting,passing,...,movement_reactions,power_shot_power,power_stamina,power_strength,power_long_shots,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness
0,93,93,78000000.0,320000.0,34,4,5,144300000.0,92.0,91.0,...,94,86,72,69,94,93,95,75,96,20
1,92,92,119500000.0,270000.0,32,4,5,197200000.0,92.0,79.0,...,93,90,76,86,87,95,81,90,88,35
2,91,91,45000000.0,270000.0,36,5,5,83300000.0,94.0,80.0,...,94,94,77,77,93,95,76,88,95,24
3,91,91,129000000.0,270000.0,29,5,5,238700000.0,83.0,86.0,...,89,80,81,53,81,86,90,93,93,35
4,91,91,125500000.0,350000.0,30,4,4,232200000.0,86.0,93.0,...,91,91,89,74,91,88,94,83,89,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,22,2,1,114000.0,35.0,46.0,...,53,46,51,46,30,38,43,42,37,38
19235,47,59,110000.0,500.0,19,2,1,193000.0,39.0,50.0,...,49,49,63,47,38,51,49,44,47,37
19236,47,55,100000.0,500.0,21,2,1,175000.0,37.0,45.0,...,46,50,56,50,34,45,46,39,36,38
19237,47,60,110000.0,500.0,19,2,1,239000.0,46.0,36.0,...,48,48,50,40,41,47,40,49,47,10


In [125]:
imp = SimpleImputer(strategy='mean')
imp.fit(df_2)
imputed_data=imp.fit_transform(df_2)
df_2=pd.DataFrame(imputed_data, columns=df_2.columns)



unique_93_overall = df_2['overall'].value_counts()[93]
if unique_93_overall == 1:
    df_2['overall'].replace(93, 92, inplace=True)

In [126]:
skill_feats = ['passing','dribbling','release_clause_eur','wage_eur',
'value_eur']
df_2['skills'] = df_2[skill_feats].mean(axis=1)
df_2.drop(columns=skill_feats, inplace=True)

shooting_feats = ['power_shot_power','shooting','power_long_shots']
df_2['shotlevel'] = df_2[shooting_feats].mean(axis=1)
df_2.drop(columns=shooting_feats, inplace=True)

passing_feats=['attacking_short_passing','skill_long_passing','skill_ball_control',
'skill_fk_accuracy']
df_2['passing']=df_2[passing_feats].mean(axis=1)
df_2.drop(columns=passing_feats,inplace=True)


movement_feats=['skill_moves','skill_dribbling','skill_curve','attacking_crossing',
'attacking_volleys','attacking_heading_accuracy','attacking_finishing']
df_2['movements']=df_2[movement_feats].mean(axis=1)
df_2.drop(columns=movement_feats,inplace=True)

mentality_feats=['mentality_composure','mentality_vision']
df_2['mentality']=df_2[mentality_feats].mean(axis=1)
df_2.drop(columns=mentality_feats,inplace=True)

power_feats=['power_strength','power_stamina']
df_2['power']=df_2[power_feats].mean(axis=1)
df_2.drop(columns=power_feats,inplace=True)

positioning_feats=['mentality_positioning',
'mentality_penalties']
df_2['power']=df_2[positioning_feats].mean(axis=1)
df_2.drop(columns=positioning_feats,inplace=True)


In [127]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  float64
 1   potential                    19239 non-null  float64
 2   age                          19239 non-null  float64
 3   international_reputation     19239 non-null  float64
 4   defending                    19239 non-null  float64
 5   physic                       19239 non-null  float64
 6   movement_reactions           19239 non-null  float64
 7   defending_marking_awareness  19239 non-null  float64
 8   skills                       19239 non-null  float64
 9   shotlevel                    19239 non-null  float64
 10  passing                      19239 non-null  float64
 11  movements                    19239 non-null  float64
 12  mentality                    19239 non-null  float64
 13  power           

In [135]:
df_2

,overall,potential,age,international_reputation,defending,physic,movement_reactions,defending_marking_awareness,skills,shotlevel,passing,movements,mentality,power
0,92.0,93.0,34.0,5.0,34.0,65.0,94.0,20.0,44524037.2,90.666667,93.00,75.857143,95.5,84.0
1,92.0,92.0,32.0,5.0,44.0,82.0,93.0,35.0,63394033.0,89.666667,82.00,73.285714,84.5,92.5
2,91.0,91.0,36.0,5.0,34.0,75.0,94.0,24.0,25714033.6,93.666667,82.25,76.000000,85.5,91.5
3,91.0,91.0,29.0,5.0,37.0,63.0,89.0,35.0,73594036.0,81.333333,87.25,72.142857,91.5,89.5
4,91.0,91.0,30.0,4.0,64.0,78.0,91.0,68.0,71610036.2,89.333333,90.25,70.000000,91.5,85.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47.0,52.0,22.0,1.0,42.0,49.0,53.0,38.0,37018.8,37.000000,46.25,33.714286,40.0,40.0
19235,47.0,59.0,19.0,1.0,41.0,51.0,49.0,37.0,60719.2,42.000000,43.50,37.285714,48.0,47.5
19236,47.0,55.0,21.0,1.0,41.0,52.0,46.0,38.0,55118.8,40.333333,46.00,33.857143,41.0,42.0
19237,47.0,60.0,19.0,1.0,15.0,42.0,48.0,10.0,69916.8,45.000000,37.50,33.142857,43.5,48.0


In [128]:
y=df_2['overall']
X=df_2.drop('overall',axis=1)
order = ['potential', 'physic', 'age', 'international_reputation', 'movement_reactions', 'defending', 'defending_marking_awareness', 'skills', 'shotlevel', 'passing', 'movements', 'mentality', 'power']
X=X[order]

In [129]:
sc=StandardScaler()
scaled=sc.fit_transform(X)
X=pd.DataFrame(scaled, columns=X.columns)
X

,potential,physic,age,international_reputation,movement_reactions,defending,defending_marking_awareness,skills,shotlevel,passing,movements,mentality,power
0,3.601780,0.019139,1.851089,10.525295,3.599846,-1.159683,-1.316900,9.741812,2.725584,2.731445,2.446327,3.341111,2.067711
1,3.437470,1.860333,1.429869,10.525295,3.489252,-0.504629,-0.574336,14.029113,2.654627,1.977180,2.259741,2.411916,2.571229
2,3.273160,1.102194,2.272309,10.525295,3.599846,-1.159683,-1.118883,5.468141,2.938455,1.994323,2.456692,2.496389,2.511991
3,3.273160,-0.197472,0.798039,10.525295,3.046874,-0.963167,-0.574336,16.346575,2.063318,2.337170,2.176814,3.003222,2.393517
4,3.273160,1.427111,1.008649,7.830520,3.268063,0.805478,1.059305,15.895806,2.630975,2.542879,2.021327,3.003222,2.156567
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,-3.134932,-1.713750,-0.676231,-0.253804,-0.934526,-0.635640,-0.425823,-0.365729,-1.082447,-0.474180,-0.611600,-1.347096,-0.538732
19235,-1.984762,-1.497139,-1.308061,-0.253804,-1.376903,-0.701145,-0.475327,-0.360344,-0.727662,-0.662746,-0.352454,-0.671319,-0.094452
19236,-2.642002,-1.388833,-0.886841,-0.253804,-1.708687,-0.701145,-0.425823,-0.361617,-0.845924,-0.491322,-0.601234,-1.262624,-0.420258
19237,-1.820452,-2.471889,-1.308061,-0.253804,-1.487498,-2.404285,-1.811942,-0.358254,-0.514791,-1.074163,-0.653063,-1.051444,-0.064833


In [130]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.1,random_state=42,stratify = y)

In [131]:
y_pred_5=bm.predict(Xtest)

In [132]:
mean_absolute_error(y_pred_5, Ytest)

1.7484746187491624

In [133]:
import pickle
filename = '/content/drive/MyDrive/AI Projects/midsem_best_model.pkl'
pickle.dump(bm, open(filename, 'wb'))

In [134]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>